## IBM Profesional Data Science Certificate
### Coursera Capstone Project
#### Yefry Lopez

 This notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
pwd

'C:\\Users\\DELL USER\\GithubProjects\\IBMDataScienceProfessionalCertificate\\Coursera_Capstone'

In [24]:
dataCR = 'SJ_Data.xlsx'
sanjose_group = pd.read_excel(dataCR)

In [25]:
sanjose_group.head(2)

,Postal Code,Borough,Neighborhood
0,10101,Carmen,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto..."
1,10102,Merced,"Bajos de la Unión, Claret, Coca Cola, Iglesias..."


### Dataframe shape

In [28]:
sanjose_group.shape

(11, 3)

### Adding Lattitude and Longitude to the dataframe from csv files

In [29]:
coordiantesSJ = 'Geospatial_Coordinates_CR_SJ.xlsx'
geo_data = pd.read_excel(coordiantesSJ)

In [30]:
#Merge two dataframes with a inner join on 'Postal Code' column
geo_SJ = pd.merge(sanjose_group, geo_data , on='Postal Code')

In [31]:
geo_SJ.shape

(11, 5)

In [32]:
geo_SJ.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,10101,Carmen,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976
1,10102,Merced,"Bajos de la Unión, Claret, Coca Cola, Iglesias...",9.94005,-84.08776
2,10103,Hospital,"Almendrares, Ángeles, Cuba, Bolívar, Carit, Co...",9.92618,-84.08926
3,10104,Catedral,"Bella Vista, Carlos María Jiménez, Catedral, D...",9.92439,-84.07122
4,10105,Zapote,"Cerrito, Jardín, Montealegre, Los Mangos, Yose...",9.92029,-84.06001
5,10106,San Francisco de Dos Ríos,"Ahogados, Cabañas, Camelias, Coopeguaria, El B...",9.90820,-84.05954
6,10107,Uruca,"Árboles, Bajo Torres, Carranza, Jardines de Au...",9.95765,-84.11571
7,10108,Mata Redonda,"Américas, Anonos, Balcón Verde, Morenos, Niza,...",9.93505,-84.11050
8,10109,Pavas,"Asturias, Aeropuerto, Bribri, Favorita Norte, ...",9.94966,-84.13615
9,10110,Hatillo,"Cañada, Belgrano, Hatillo Centro, Hatillo 1, H...",9.91804,-84.10004


### k-means Clustering Analysis

#### Segmenting and Clustering Neighborhoods in Toronto City

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [33]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Quickly examine the resulting dataframe.

In [34]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(geo_SJ['Borough'].unique()),
        geo_SJ.shape[0]
    )
)

The dataframe has 11 boroughs and 11 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto City

In [37]:
address = 'San Jose , Costa Rica'

geolocator = Nominatim(user_agent="sanjose_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 9.9325427, -84.0795782.


#### Create a map of New York with neighborhoods superimposed on top.

In [38]:
#create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_SJ['Latitude'], geo_SJ['Longitude'], geo_SJ['Borough'], geo_SJ['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in 'Scarborough'. So let's slice the original dataframe and create a new dataframe of the 'Scarborough' data.

In [39]:
scarborough_data = geo_SJ[geo_SJ['Borough'] == 'Carmen'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,10101,Carmen,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976


Let's get the geographical coordinates of Scarborough.

In [40]:
address = 'Carmen, San Jose, Costa Rica'

geolocator = Nominatim(user_agent="sanjose_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 9.9325427, -84.0795782.


As we did with all of Toronto City, let's visualizat Scarborough the neighborhoods in it.

In [20]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [41]:
CLIENT_ID = 'W20G0AZ0L3UKPMOQY0DFYENROKAXDBFEGRYRLNPPBRAYR5ML'
CLIENT_SECRET =  'US32PLWNVEJENEYNFVP20CUEXNKZ4JYBPGBAD3XUAVDEARYM'
VERSION = '20180605'
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W20G0AZ0L3UKPMOQY0DFYENROKAXDBFEGRYRLNPPBRAYR5ML
CLIENT_SECRET:US32PLWNVEJENEYNFVP20CUEXNKZ4JYBPGBAD3XUAVDEARYM


#### Let's explore the first neighborhood in our dataframe.

In [42]:
scarborough_data.loc[0, 'Neighborhood']

'Amón, Aranjuez, Carmen, Empalme, Escalante,Otoya,Barrio La California'

#### Get the neighborhood's latitude and longitude values.

In [43]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Amón, Aranjuez, Carmen, Empalme, Escalante,Otoya,Barrio La California are 9.93623, -84.06976.


#### Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [44]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=W20G0AZ0L3UKPMOQY0DFYENROKAXDBFEGRYRLNPPBRAYR5ML&client_secret=US32PLWNVEJENEYNFVP20CUEXNKZ4JYBPGBAD3XUAVDEARYM&v=20180605&ll=9.93623,-84.06976&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fef50f5499ac81c60f17c2e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'El Carmen',
  'headerFullLocation': 'El Carmen, San José',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 94,
  'suggestedBounds': {'ne': {'lat': 9.940730004500004,
    'lng': -84.06519999939668},
   'sw': {'lat': 9.931729995499996, 'lng': -84.07432000060332}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c8d536c8018a1cd3786f4d2',
       'name': 'Parque Nacional',
       'location': {'address': 'Paseo Las Damas',
        'crossStreet': 'Av. 3, calle 13 y 17',
        'lat': 9.934823643484291,
        'lng': -84.07056856241404,
        'labeledLatLng

#### Function that extracts the category of the venue

In [46]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Parque Nacional,Park,9.934824,-84.070569
1,Maza Bistró,Bistro,9.934567,-84.069782
2,La Ventanita Meraki,Restaurant,9.934628,-84.069178
3,La Terrasse,French Restaurant,9.937611,-84.071267
4,Museo de Arte y Diseño Contemporáneo,Art Museum,9.935413,-84.071812


And how many venues were returned by Foursquare?

In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

94 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Malvern, Rouge

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Amón, Aranjuez, Carmen, Empalme, Escalante,Otoya,Barrio La California


#### Let's check the size of the resulting dataframe

In [51]:
print(scarborough_venues.shape)
scarborough_venues.head()

(94, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976,Parque Nacional,9.934824,-84.070569,Park
1,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976,Maza Bistró,9.934567,-84.069782,Bistro
2,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976,La Ventanita Meraki,9.934628,-84.069178,Restaurant
3,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976,La Terrasse,9.937611,-84.071267,French Restaurant
4,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",9.93623,-84.06976,Museo de Arte y Diseño Contemporáneo,9.935413,-84.071812,Art Museum


Neighborhood              0
Neighborhood Latitude     0
Neighborhood Longitude    0
Venue                     0
Venue Latitude            0
Venue Longitude           0
Venue Category            0
dtype: int64

#### Let's check how many venues were returned for each neighborhood

In [52]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Amón, Aranjuez, Carmen, Empalme, Escalante,Otoya,Barrio La California",94,94,94,94,94,94


#### Let's find out how many unique categories can be curated from all the returned venues

In [53]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 49 uniques categories.


### 3. Analyze Each Neighborhood

In [54]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Yoga Studio,Art Museum,Asian Restaurant,Bakery,Bar,Bistro,Bookstore,Boutique,Buffet,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cultural Center,Event Space,Fast Food Restaurant,Food Court,French Restaurant,Gift Shop,Grocery Store,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Juice Bar,Latin American Restaurant,Market,Movie Theater,Museum,Music Store,Neighborhood,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Plaza,Pub,Public Art,Restaurant,Rock Club,Salad Place,Theater,Train Station
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",0,0,0,0,0,0,0,0,0,0,0,0,0


#### And let's examine the new dataframe size.

In [55]:
scarborough_onehot.shape

(94, 49)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [56]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Yoga Studio,Art Museum,Asian Restaurant,Bakery,Bar,Bistro,Bookstore,Boutique,Buffet,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cultural Center,Event Space,Fast Food Restaurant,Food Court,French Restaurant,Gift Shop,Grocery Store,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Juice Bar,Latin American Restaurant,Market,Movie Theater,Museum,Music Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Plaza,Pub,Public Art,Restaurant,Rock Club,Salad Place,Theater,Train Station
0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",0.010638,0.021277,0.010638,0.010638,0.170213,0.021277,0.010638,0.021277,0.010638,0.021277,0.010638,0.010638,0.074468,0.010638,0.010638,0.010638,0.010638,0.021277,0.031915,0.021277,0.010638,0.021277,0.010638,0.010638,0.021277,0.010638,0.021277,0.010638,0.021277,0.021277,0.010638,0.010638,0.021277,0.021277,0.010638,0.010638,0.010638,0.031915,0.010638,0.021277,0.021277,0.010638,0.010638,0.042553,0.010638,0.010638,0.031915,0.010638


#### Let's confirm the new size

In [57]:
scarborough_grouped.shape

(1, 49)

#### Let's print each neighborhood along with the top 5 most common venues

In [58]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amón, Aranjuez, Carmen, Empalme, Escalante,Otoya,Barrio La California----
         venue  freq
0          Bar  0.17
1  Coffee Shop  0.07
2   Restaurant  0.04
3         Park  0.03
4      Theater  0.03




#### Let's put that into a _pandas_ dataframe

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Amón, Aranjuez, Carmen, Empalme, Escalante,Oto...",Bar,Coffee Shop,Restaurant,Food Court,Theater,Park,Movie Theater,Café,French Restaurant,Grocery Store


### 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [61]:
#Run k-means to cluster the neighborhood into 5 clusters

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

ValueError: n_samples=1 should be >= n_clusters=5

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how ='inner')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Moving Target,Bar,Home Service,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Bank,Medical Center,Restaurant,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Breakfast Spot,Department Store,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store


In [44]:
scarborough_merged.isnull().sum()

Postal Code               0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

#### Finally, let's visualize the resulting clusters

In [45]:

#visualize the resulting clusters

#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       


In [46]:
map_clusters

### 5. Examine Clusters

#### Cluster 1

In [48]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,0,Skating Rink,General Entertainment,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 2

In [51]:

scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,Scarborough,1,Moving Target,Bar,Home Service,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,1,Bank,Medical Center,Restaurant,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Breakfast Spot,Department Store,College Stadium
4,Scarborough,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store
6,Scarborough,1,Discount Store,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,1,Bakery,Bus Line,Ice Cream Shop,Bus Station,Park,Metro Station,Intersection,Soccer Field,Fast Food Restaurant,Coffee Shop
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Brewery,Pet Store,Chinese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint
11,Scarborough,1,Middle Eastern Restaurant,Sandwich Place,Auto Garage,Bakery,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,Scarborough,1,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1,Fast Food Restaurant,Pizza Place,Pharmacy,Noodle House,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Italian Restaurant,Chinese Restaurant


#### Cluster 3

In [52]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 4

In [53]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 5

In [54]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4,American Restaurant,Motel,Home Service,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
